In [ ]:
import numpy as np

# Introduction to Variational Quantum Eigensolver

In this tutorial we will learn an introduction to variational quantum eigensolver for calculation of lowest eigenvalue of a Hamiltonan or Hermitian matrix.

In VQE, a quantum computer and a classical computer work together to calculate the ground state energy of a molecule.

# Let's review some stuffs first

## What is Hermitian matrix ?

A Hermitian matrix is a square matrix $\mathbf{A}$ that satisfies:
$$
\mathbf{A} = \mathbf{A}^{\dagger}
$$
where $\mathbf{A}^{\dagger}$ is conjugate transpose of $\mathbf{A}$.
For a real matrix, a Hermitian matrix is simply a symmetric matrix, i.e. a matrix which is equal to its transpose.

Hamiltonian matrix which is found in electronic structure problem (chemistry, material science, etc.) is an example of Hermitian matrix. 

Let's create a random matrix first.

In [ ]:
A = np.random.rand(4,4)
A

### Question

Is matrix `A` Hermitian?

In [ ]:
A.T - A

We can "make" `A` Hermitian or symmetric by taking only its lower triangule part or upper triangle part.
Alternatively we can do something like this:

In [ ]:
A = 0.5*(A.T + A)
A

In [ ]:
A.T - A

## Eigenvalue equation

An **eigenvalue** $\lambda$ of a matrix $\mathbf{A}$ obeys the following equation:
$$
\mathbf{A} \mathbf{x} = \lambda \mathbf{x}
$$
for some vector $\mathbf{x}$. The vector $\mathbf{x}$ is also called an **eigenvector**.

This eigenvalue equation appears in many applications. For quantum chemistry, eigenvalues are the energies of a system. We usually only concern ourselves with the lowest eigenvalue or the lowest energy.

In [ ]:
λ = np.linalg.eigvalsh(A)

In [ ]:
λ

In [ ]:
λ, X = np.linalg.eigh(A)

In [ ]:
λ[0]

In [ ]:
x0 = X[:,0]

In [ ]:
A @ x0

In [ ]:
λ[0] * x0

In [ ]:
np.dot(x0, x0)

## Optimization

Optimizing a scalar function

# Qiskit

There are several things that we need to prepare before executing a VQE calculation.

- The Hamiltonian matrix needs to be represented as tensor products of Pauli
- Parameterized ansatz (recipe) for quantum circuit
- minization algorithm

To create Pauli representation of a matrix, we can use `SparsePauliOp` class from Qiskit.

In [ ]:
from qiskit.quantum_info.operators import SparsePauliOp

In [ ]:
np.random.seed(1234)
A = np.random.rand(4,4) + 10*np.eye(4) # create a digonally dominant matrix
A = 0.5*(A + A.T)
A

In [ ]:
pauli_op = SparsePauliOp.from_operator(A)

In [ ]:
pauli_op

## Checking Pauli representation

In [ ]:
pauli_op_list = pauli_op.to_list()
pauli_op_list

In [ ]:
import gate_matrix

In [ ]:
def process_gate_str(g_str):
    Nqubit = len(g_str)
    matRes = np.eye(Nqubit, dtype=np.complex128)
    mat_list = []
    for s in g_str:
        if s == 'X':
            mat_list.append(gate_matrix.X)
        elif s == 'Y':
            mat_list.append(gate_matrix.Y)
        elif s == 'Z':
            mat_list.append(gate_matrix.Z)
        elif s == 'I':
            mat_list.append(gate_matrix.I)
        else:
            raise RuntimeError(f"Unknown gate: {s}")
    Nmat = len(mat_list)
    if Nmat >= 2:
        A = np.kron(mat_list[0], mat_list[1])
        for i in range(2,Nmat):
            A = np.kron(A, mat_list[2])
    else:
        A = mat_list[0]
    return A

In [ ]:
matA = np.zeros( (4,4), dtype=np.complex128)
for p in pauli_op_list:
    matA += p[1] * process_gate_str(p[0])

In [ ]:
np.real(matA)

Original matrix:

In [ ]:
A

## Qiskit: built in algorithm

In [ ]:
ref_value = np.linalg.eigvalsh(pauli_op.to_matrix())[0]
print(f"Reference value: {ref_value:.5f}")

In [ ]:
# define ansatz and optimizer
from qiskit.circuit.library import EfficientSU2
from qiskit_algorithms.optimizers import SPSA

In [ ]:
import qiskit.circuit.library
import qiskit_algorithms.optimizers

In [ ]:
iterations = 125
ansatz = EfficientSU2(pauli_op.num_qubits)
spsa = SPSA(maxiter=iterations)

In [ ]:
# define callback
# note: Re-run this cell to restart lists before training
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

In [ ]:
from qiskit_algorithms.utils import algorithm_globals
seed = 1234
algorithm_globals.random_seed = seed

In [ ]:
Nshots = 2**10
# define Aer Estimator for noiseless statevector simulation
from qiskit_aer.primitives import Estimator as AerEstimator
noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": Nshots},
    transpile_options={"seed_transpiler": seed},
)

In [ ]:
# instantiate and run VQE
from qiskit_algorithms import VQE

In [ ]:
vqe = VQE(noiseless_estimator, ansatz, optimizer=spsa, callback=store_intermediate_result)

In [ ]:
result = vqe.compute_minimum_eigenvalue(operator=pauli_op)

print(f"VQE on Aer qasm simulator (no noise): {result.eigenvalue.real:.5f}")
print(f"Delta from reference energy value is {(result.eigenvalue.real - ref_value):.5f}")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.style.use("dark_background")

In [ ]:
plt.plot(counts, values);